<a href="https://colab.research.google.com/github/giyuhwang/waste_classification/blob/main/APS360_Team_3_Project_with_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

APS360 Team 3 Project with Github

Github Link: https://github.com/giyuhwang/waste_classification 